In [ ]:
import pandas as pd

def build_feature_dataset(
    input_paths: list[str],
    output_path: str,
    region: str,
    cols: list[str],
    freq: str = "30T",
    plot: bool = True
) -> tuple[pd.DataFrame, pd.Series]:
    """
    Loading Features 
    Args:
        input_paths:    List of Parquet file paths. Each file must load into a DataFrame
                        whose columns are a MultiIndex with levels [region, variable_name].
        output_path:    File‐path (including filename) where the feature report should be written.
        region:         The first‐level column key (region) to subset by after concatenation.
        cols:           A list of variable names (second‐level columns) to keep, once we subset to `region`.
        freq:           A Pandas offset alias (e.g. "30T", "15T", "1H") used to resample each DataFrame.
                        Default is "30T".
        plot:           If True, calls `generate_feature_report(...)` on the final feature set.

    Returns:
        feat:   A DataFrame of shape [n_samples × n_features], containing:
                • the time‐features (weekday, hour, month, etc.),
                • the chosen columns in `cols`,
                • and any newly added columns (forward‐filled) for modeling.
        tar:    A pd.Series named "Imbalance_Minus_Spot", aligned with `feat.index`, 
                containing the (imbalance_price − spot_price) at each timestamp.
    """
    # ----------------------------------------------------------------------------------
    # Helpers for timezone‐normalization + resampling
    # ----------------------------------------------------------------------------------
    def _load_and_resample_one(path: str, freq_rule: str) -> pd.DataFrame:
        """
        Loads one Parquet file into a DataFrame with a DateTimeIndex, normalizes
        its index to Asia/Tokyo, and resamples to `freq_rule` using .mean().
        """
        df = pd.read_parquet(path)

        # Ensure index is datetime:
        df = df.copy()
        df.index = pd.to_datetime(df.index)

        # If tz‐naive → assume it's already JST, so localize to Asia/Tokyo.
        # If tz‐aware (e.g. UTC or anything), convert to Asia/Tokyo.
        if df.index.tz is None:
            df.index = df.index.tz_localize("Asia/Tokyo")
        else:
            df.index = df.index.tz_convert("Asia/Tokyo")

        # Resample to the requested frequency, taking the mean of each
        # (e.g. if `freq_rule="30T"`, each 30‐minute block is averaged).
        df_resampled = df.resample(freq_rule).mean()
        return df_resampled

    # ----------------------------------------------------------------------------------
    # 1) Load + resample each input DataFrame; collect start/end times
    # ----------------------------------------------------------------------------------
    loaded_dfs = []
    start_times = []
    end_times = []

    for path in input_paths:
        df_resampled = _load_and_resample_one(path, freq)
        loaded_dfs.append(df_resampled)

        # Record the new index range
        start_times.append(df_resampled.index.min())
        end_times.append(df_resampled.index.max())

    if not loaded_dfs:
        raise ValueError("`input_paths` must contain at least one parquet file.")

    # ----------------------------------------------------------------------------------
    # 2) Find the common date‐range: [latest_start, earliest_end]
    # ----------------------------------------------------------------------------------
    latest_start = max(start_times)
    earliest_end = min(end_times)

    if latest_start >= earliest_end:
        raise ValueError(
            f"No overlapping time‐range found among the loaded files. "
            f"latest_start={latest_start}, earliest_end={earliest_end}"
        )

    # 3) Truncate each DataFrame to [latest_start : earliest_end]
    aligned_dfs = [
        df.loc[latest_start : earliest_end] for df in loaded_dfs
    ]

    # ----------------------------------------------------------------------------------
    # 4) Concatenate side‐by‐side (axis=1)
    # ----------------------------------------------------------------------------------
    # Since each df had columns = MultiIndex [region, variable], 
    # the concatenation keeps the same MultiIndex column structure.
    concatenated = pd.concat(aligned_dfs, axis=1)

    # ----------------------------------------------------------------------------------
    # 5) Subset by region (first level) and then by `cols` (second level)
    # ----------------------------------------------------------------------------------
    # This picks out one “slice” of the MultiIndex at level=0 == region.
    try:
        df_region = concatenated[region]
    except KeyError:
        raise KeyError(f"Region '{region}' not found in the concatenated columns.")

    # Now df_region’s columns are the second level only. We keep exactly `cols`.
    missing = [c for c in cols if c not in df_region.columns]
    if missing:
        raise KeyError(f"The following requested columns are not present for region {region}: {missing}")

    df_region = df_region[cols]

    # ----------------------------------------------------------------------------------
    # 6) Find the first+last index where BOTH spot & imbalance are non‐NaN
    # ----------------------------------------------------------------------------------
    imb_col = "pri_imb_down_%_kwh_jst_min30_a"
    spot_col = "pri_spot_jepx_%_kwh_jst_min30_a"

    # Ensure those two are in `cols` (or else we can’t form the target)
    if imb_col not in df_region.columns or spot_col not in df_region.columns:
        raise KeyError(
            f"Cannot find both target columns ('{imb_col}' and '{spot_col}') in df_region. "
            f"Got columns={list(df_region.columns)}"
        )

    # Build a mask where both are non‐NaN:
    both_valid = (
        df_region[imb_col].notna() &
        df_region[spot_col].notna()
    )
    # If there is no timestamp where both are valid, it's an error:
    if not both_valid.any():
        raise ValueError(
            f"No timestamp exists where both '{imb_col}' and '{spot_col}' are non‐NaN."
        )

    valid_times = df_region.index[both_valid]
    crop_start = valid_times.min()
    crop_end = valid_times.max()

    # Crop the DataFrame so that the first row has both non‐NaN, and the last row has both non‐NaN
    df_region = df_region.loc[crop_start : crop_end]

    # ----------------------------------------------------------------------------------
    # 7) Forward‐fill any remaining NaNs (limit=1)
    # ----------------------------------------------------------------------------------
    df_region = df_region.ffill(limit=1)

    # ----------------------------------------------------------------------------------
    # 8) Construct time‐features
    #    (Assumes you already have a function `construct_time_features(df)` defined elsewhere.)
    # ----------------------------------------------------------------------------------
    construct_time_features(df_region)

    # ----------------------------------------------------------------------------------
    # 9) Create the target series: "Imbalance_Minus_Spot"
    # ----------------------------------------------------------------------------------
    tar = df_region[imb_col] - df_region[spot_col]
    tar.name = "Imbalance_Minus_Spot"

    # ----------------------------------------------------------------------------------
    # 10) Optionally generate a feature report
    #    (Assumes you already have `generate_feature_report(...)` imported.)
    # ----------------------------------------------------------------------------------
    if plot:
        # name="Features" is arbitrary; you can change if you like
        generate_feature_report(
            features=df_region,
            target=tar,
            document_name=output_path,
            name="Features"
        )

    # ----------------------------------------------------------------------------------
    # 11) Return the final feature‐DataFrame and the target‐Series
    # ----------------------------------------------------------------------------------
    return df_region, tar

In [ ]:
import pandas as pd

def load_and_encode_imbalance_cs(
    imbalance_path: str,
    region: str,
    timestamp_col: str = "timestamp"
) -> pd.DataFrame:
    """
    Load the imbalance_cs_train Parquet, filter to the rows where `zone == region`,
    and then create eight one-hot (dummy) columns indicating which other zones share
    the same `wide_area_category` block code at each timestamp.

    Args:
        imbalance_path:   Path to the Parquet file containing imbalance_cs_train data.
                          It is assumed to have columns:
                            - timestamp_col  (DatetimeIndex)
                            - "zone"         (string: one of the nine region names)
                            - "wide_area_category" (int: block code)
                            - …any number of other features…
        region:           The name of the zone you want to keep (e.g. "tokyo", "kansai", etc.)
        timestamp_col:    The name of the timestamp column in the file. After loading,
                          this column will be converted to a DateTimeIndex. Default "timestamp".
                          If you actually have separate "date" + "period" columns, see note below.

    Returns:
        df_region:  A DataFrame indexed by timestamp (tz-aware if the file was),
                    containing:
                      • all original columns from the imbalance file for rows where zone=region
                        (EXCEPT "zone" and "wide_area_category", which we drop once we extract them),
                      • plus eight new columns of the form "is_same_block_<zone_name>" (int),
                        giving 1 if that other zone shares the same wide_area_category code at that time,
                        else 0.

        Example columns:
            [ ... other tokyo features ..., 
              is_same_block_hokkaido,
              is_same_block_tohoku,
              is_same_block_chubu,
              is_same_block_hokuriku,
              is_same_block_kansai,
              is_same_block_chugoku,
              is_same_block_shikoku,
              is_same_block_kyushu,
              is_same_block_okinawa
            ]
    """
    # ----------------------------------------
    # 1) Read the Parquet
    # ----------------------------------------
    df = pd.read_parquet(imbalance_path)

    # ----------------------------------------
    # 2) Parse/normalize the timestamp index
    #    (If your file truly has a single datetime column:)
    # ----------------------------------------
    if timestamp_col not in df.columns:
        # If instead your file has 'date' + 'period' (30-minute slot):
        # uncomment + adjust the following as needed:
        #
        # df["datetime_jst"] = (
        #     pd.to_datetime(df["date"].astype(str))
        #     + pd.to_timedelta((df["period"] - 1) * 30, unit="m")
        # )
        # df["datetime_jst"] = df["datetime_jst"].dt.tz_localize("Asia/Tokyo")
        # df = df.set_index("datetime_jst")
        # 
        # In that case, just reassign timestamp_col = "datetime_jst":
        # timestamp_col = "datetime_jst"
        #
        # For now, I’ll raise an error so you can correct to your actual schema:
        raise KeyError(
            f"Column '{timestamp_col}' not found in {imbalance_path}. "
            f"Either rename your datetime column to '{timestamp_col}', or supply "
            f"‘date’ + ‘period’ parsing logic above."
        )
    else:
        # If tz information is missing, you may need to localize → Asia/Tokyo.
        df[timestamp_col] = pd.to_datetime(df[timestamp_col])
        if df[timestamp_col].dt.tz is None:
            df[timestamp_col] = df[timestamp_col].dt.tz_localize("Asia/Tokyo")
        else:
            df[timestamp_col] = df[timestamp_col].dt.tz_convert("Asia/Tokyo")

        df = df.set_index(timestamp_col)

    # ----------------------------------------
    # 3) Pivot out the “wide_area_category” codes by zone
    #    so we can quickly see “at time t, zone X had code Y.”
    # ----------------------------------------
    # We only need “zone” and “wide_area_category” for this step.
    # If there are multiple rows for (timestamp, zone), you might want to
    # take the latest or drop duplicates first. Here, I’ll assume it’s unique.
    pivot_block = df[["zone", "wide_area_category"]].copy()
    # Make sure “zone” is a column, not the index:
    pivot_block = pivot_block.reset_index()  

    # Create a DataFrame whose index is timestamp, columns are the 9 zone names,
    # and values are the wide_area_category for that zone at that timestamp:
    block_df = pivot_block.pivot(
        index=timestamp_col,
        columns="zone",
        values="wide_area_category"
    )

    # ----------------------------------------
    # 4) Filter to just the “region” rows
    # ----------------------------------------
    # This gives us one row per timestamp for our region. If the original file
    # had multiple (timestamp, region) rows, you could .drop_duplicates(...) first.
    df_region = df[df["zone"] == region].copy()

    # If region never appears, we must error:
    if df_region.empty:
        raise KeyError(f"No rows found where zone == '{region}' in {imbalance_path}")

    # We’ll want to drop “zone” and “wide_area_category” from df_region once we extract them.
    # First, record the region’s block code (so we can compare to others):
    df_region["region_block_code"] = df_region["wide_area_category"]

    # ----------------------------------------
    # 5) Build the dummy columns
    # ----------------------------------------
    # For each timestamp t, block_df.loc[t] is a row whose columns are the 9 zone names,
    # and whose values are that zone’s wide_area_category code at time t.
    #
    # We want a boolean DataFrame: “is zone Z in the same block as our region at time t?”
    # That is: block_df.eq(region_block_code, axis=0).
    region_codes = df_region["region_block_code"].rename("region_block_code")

    # Align the index of block_df with the index of df_region (some timestamps might not match exactly)
    # We'll reindex block_df to only those timestamps where region appears.
    block_df_at_region_times = block_df.reindex(df_region.index)

    # Now compare: a True wherever block_df code == region_block_code
    same_block_bool = block_df_at_region_times.eq(region_codes, axis=0)

    # Convert True/False → 1/0
    same_block_int = same_block_bool.astype(int)

    # We do not need a dummy for the region itself (since it is obviously 1),
    # so drop that column if you like, or keep it. I’ll drop it to get exactly 8 columns:
    if region in same_block_int.columns:
        same_block_int = same_block_int.drop(columns=[region])

    # Rename the columns to “is_same_block_<zone>”
    same_block_int.columns = [f"is_same_block_{z}" for z in same_block_int.columns]

    # ----------------------------------------
    # 6) Merge these dummy columns back onto df_region
    # ----------------------------------------
    df_region = pd.concat([df_region, same_block_int], axis=1)

    # ----------------------------------------
    # 7) Drop the helper columns “zone” + “wide_area_category” + “region_block_code”
    #    (unless you want to keep them for reference)
    # ----------------------------------------
    df_region = df_region.drop(
        columns=["zone", "wide_area_category", "region_block_code"],
        errors="ignore"
    )

    # Now df_region has:
    #   • its original features (all columns except we dropped zone/wide_area_category),
    #   • plus exactly eight new columns “is_same_block_<other_zone>”.

    return df_region

In [ ]:
import pandas as pd

def load_and_encode_zone_data(path: str, region: str, timestamp_col: str = "timestamp") -> pd.DataFrame:
    df = pd.read_parquet(path)

    # Parse timestamp
    if timestamp_col not in df.columns:
        raise KeyError(f"Column '{timestamp_col}' not found in {path}")
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    if df[timestamp_col].dt.tz is None:
        df[timestamp_col] = df[timestamp_col].dt.tz_localize("Asia/Tokyo")
    else:
        df[timestamp_col] = df[timestamp_col].dt.tz_convert("Asia/Tokyo")
    df = df.set_index(timestamp_col)

    # Pivot block codes by zone
    pivot = df[["zone", "wide_area_category"]].reset_index()
    block_df = pivot.pivot(index=timestamp_col, columns="zone", values="wide_area_category")

    # Filter to region
    df_region = df[df["zone"] == region].copy()
    if df_region.empty:
        raise KeyError(f"No rows found with zone == '{region}' in {path}")
    df_region["region_block_code"] = df_region["wide_area_category"]

    # Create dummy columns
    block_df_region = block_df.reindex(df_region.index)
    same_block = block_df_region.eq(df_region["region_block_code"], axis=0).astype(int)
    if region in same_block.columns:
        same_block = same_block.drop(columns=[region])
    same_block.columns = [f"is_same_block_{z}" for z in same_block.columns]

    # Merge and drop helpers
    df_region = pd.concat([df_region, same_block], axis=1)
    df_region = df_region.drop(columns=["zone", "wide_area_category", "region_block_code"], errors="ignore")

    return df_region


def build_feature_dataset(
    input_paths: list[str],
    imbalance_path: str,
    daily_occto_path: str,
    output_path: str,
    region: str,
    cols: list[str],
    freq: str = "30T",
    plot: bool = True
) -> tuple[pd.DataFrame, pd.Series]:
    # Load and process imbalance and daily_occto
    df_imb = load_and_encode_zone_data(imbalance_path, region, timestamp_col="timestamp")
    df_imb = df_imb.resample(freq).mean()

    df_daily = load_and_encode_zone_data(daily_occto_path, region, timestamp_col="timestamp")
    df_daily = df_daily.resample(freq).mean()

    # Helper for other files
    def _load_resample(path: str) -> pd.DataFrame:
        df = pd.read_parquet(path).copy()
        df.index = pd.to_datetime(df.index)
        if df.index.tz is None:
            df.index = df.index.tz_localize("Asia/Tokyo")
        else:
            df.index = df.index.tz_convert("Asia/Tokyo")
        return df.resample(freq).mean()

    loaded_dfs = [df_imb, df_daily]
    start_times = [df_imb.index.min(), df_daily.index.min()]
    end_times = [df_imb.index.max(), df_daily.index.max()]

    for path in input_paths:
        df_r = _load_resample(path)
        loaded_dfs.append(df_r)
        start_times.append(df_r.index.min())
        end_times.append(df_r.index.max())

    if not loaded_dfs:
        raise ValueError("No data loaded.")

    # Find common range
    latest_start = max(start_times)
    earliest_end = min(end_times)
    if latest_start >= earliest_end:
        raise ValueError(f"No overlapping range: {latest_start} >= {earliest_end}")

    aligned = [df.loc[latest_start:earliest_end] for df in loaded_dfs]

    # Concatenate
    concatenated = pd.concat(aligned, axis=1)

    # Subset to region and cols
    try:
        df_region_multi = concatenated[region]
    except KeyError:
        raise KeyError(f"Region '{region}' not found.")

    missing = [c for c in cols if c not in df_region_multi.columns]
    if missing:
        raise KeyError(f"Missing columns for region {region}: {missing}")
    df_region_multi = df_region_multi[cols]

    # Crop to valid target rows
    imb_col = "pri_imb_down_%_kwh_jst_min30_a"
    spot_col = "pri_spot_jepx_%_kwh_jst_min30_a"
    if imb_col not in df_region_multi.columns or spot_col not in df_region_multi.columns:
        raise KeyError(f"Target columns missing: {imb_col}, {spot_col}")
    mask = df_region_multi[imb_col].notna() & df_region_multi[spot_col].notna()
    if not mask.any():
        raise ValueError("No valid target rows.")
    valid_idx = df_region_multi.index[mask]
    df_region_multi = df_region_multi.loc[valid_idx.min():valid_idx.max()]

    # Forward-fill
    df_region_multi = df_region_multi.ffill(limit=1)

    # Time features
    construct_time_features(df_region_multi)

    # Target series
    tar = df_region_multi[imb_col] - df_region_multi[spot_col]
    tar.name = "Imbalance_Minus_Spot"

    # Feature report
    if plot:
        generate_feature_report(
            features=df_region_multi,
            target=tar,
            document_name=output_path,
            name="Features"
        )

    return df_region_multi, tar


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# -------------------------------------------------------------------
# 1) Standardization / Un‐standardization Helpers
# -------------------------------------------------------------------

def identity_standardize(df: pd.DataFrame, features: list, target: str):
    """
    Shapes data and adds intercept without scaling.

    Args:
        df: pandas DataFrame (length T)
        features: list of feature column names (to include in X)
        target: name of target column

    Returns:
        x: np.ndarray of shape (K, T, 1),
           where K = 1 + len(features);  first row is intercept,
           remaining rows correspond to features in the given order.
        y: np.ndarray of shape (T, 1)
        means: np.ndarray of shape (q,) (all zeros if no numeric features)
        stds: np.ndarray of shape (q,) (all ones if no numeric features)
    """
    # 0) Check all requested features exist
    missing = [c for c in features if c not in df.columns]
    if missing:
        raise ValueError(f"Requested features not found in DataFrame: {missing}")

    T = df.shape[0]
    N = 1

    # 1) Build feature array in THE ORDER specified by `features`
    feat_array = df[features].to_numpy()        # shape = (T, K-1)
    feat_TN    = feat_array.T[:, :, np.newaxis] # shape = (K-1, T, 1)

    # 2) Add intercept
    inter = np.ones((1, T, N))
    x     = np.concatenate((inter, feat_TN), axis=0)  # shape = (K, T, 1)

    # 3) Build target vector (unchanged)
    y = df[target].to_numpy().reshape(T, N)  # shape = (T, 1)

    # 4) Identify numeric features so we can build placeholder means/stds
    numeric_cols = [c for c in features if pd.api.types.is_numeric_dtype(df[c])]
    q = len(numeric_cols)

    # Return dummy means/stds so that downstream code can always unpack 4 items
    means = np.zeros(q)
    stds  = np.ones(q)

    return x, y, means, stds


def z_standardize(df: pd.DataFrame, features: list, target: str):
    """
    Takes in a DataFrame with specified features, adds an intercept, and returns:
      - x: (K, T, 1) with intercept + standardized numeric + raw categorical
      - y: (T, 1) the target vector (unchanged)
      - means: (q,)  means of numeric columns
      - stds:  (q,)  stds of numeric columns

    Args:
        df: pandas DataFrame (length T)
        features: list of feature column names (some numeric, some categorical)
        target: name of target column

    Returns:
        x: np.ndarray of shape (K, T, 1), where
           K = 1 + len(features),
           row 0 = intercept (all ones),
           rows 1..q = (X_numeric - mean)/std,
           rows q+1.. = categorical columns (unchanged).
        y: np.ndarray of shape (T, 1)
        means: np.ndarray of shape (q,), where q = number of numeric features
        stds:  np.ndarray of shape (q,)
    """
    # 0) Check all features exist
    missing = [c for c in features if c not in df.columns]
    if missing:
        raise ValueError(f"Requested features not found in DataFrame: {missing}")

    T = df.shape[0]
    N = 1

    # 1) Partition features into numeric vs. categorical (preserving order)
    numeric_cols = [c for c in features if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols     = [c for c in features if not pd.api.types.is_numeric_dtype(df[c])]
    cols_order   = numeric_cols + cat_cols  # final order for columns

    # 2) Compute means/stds on numeric columns
    if len(numeric_cols) > 0:
        means = df[numeric_cols].mean().to_numpy()       # shape = (q,)
        stds  = df[numeric_cols].std().to_numpy() + 1e-8 # shape = (q,)
    else:
        means = np.zeros(0)
        stds  = np.ones(0)

    # 3) Standardize numeric columns
    if len(numeric_cols) > 0:
        df_scaled_numeric = (df[numeric_cols] - means) / stds  # DataFrame shape = (T, q)
    else:
        # Create an empty DataFrame with T rows and 0 columns
        df_scaled_numeric = pd.DataFrame(index=df.index)

    # 4) Combine scaled numerics with categorical, in the order specified
    df_combined = pd.concat([df_scaled_numeric, df[cat_cols]], axis=1)
    df_new      = df_combined[cols_order]  # re‐order (though concat already did)

    # 5) Build feature‐tensor x: shape = (K, T, 1)
    feat_array = df_new.to_numpy()               # shape = (T, K-1)
    feat_TN    = feat_array.T[:, :, np.newaxis]  # shape = (K-1, T, 1)
    inter      = np.ones((1, T, N))              # shape = (1, T, 1)
    x          = np.concatenate((inter, feat_TN), axis=0)  # shape = (K, T, 1)

    # 6) Build target vector (unchanged)
    y = df[target].to_numpy().reshape(T, N)  # shape = (T, 1)

    return x, y, means, stds


def z_unstandardize(betas: np.ndarray, means: np.ndarray, stds: np.ndarray):
    """
    Un‐scale time‐varying betas from Z‐score standardization.

    Args:
        betas:   np.ndarray of shape (K, T, 1),
                 where K = 1 + q + (#categorical features),
                 row 0 = scaled intercept,
                 rows 1..q = scaled numeric slopes,
                 rows q+1.. = categorical slopes (unchanged),
                 T = number of timepoints.
        means:   np.ndarray of shape (q,), mean for each numeric predictor
        stds:    np.ndarray of shape (q,), std for each numeric predictor

    Returns:
        betas_unscaled: np.ndarray of shape (K, T, 1),
                        with numeric slopes and intercept un‐scaled back to original units.
    """
    q = means.shape[0]  # number of numeric features
    T = betas.shape[1]
    N = 1

    # 1) Un‐scale numeric slopes
    #    betas[1:q+1, t, 0] is the scaled slope; divide by stds[i]
    if q > 0:
        slopes_scaled = betas[1 : q+1, :, :]                     # shape = (q, T, 1)
        slopes_unscaled = slopes_scaled / stds.reshape(q, 1, 1)  # shape = (q, T, 1)
    else:
        slopes_unscaled = np.zeros((0, T, N))

    # 2) Adjust intercept
    #    intercept_unscaled[t] = scaled_intercept[t] - sum_i (scaled_slope_i[t] * (means[i]/stds[i]))
    if q > 0:
        temp = (means / stds).reshape(q, 1, 1)  # shape = (q, 1, 1)
        adjustment = np.sum(betas[1 : q+1, :, :] * temp, axis=0)  # shape = (T, 1)
    else:
        adjustment = np.zeros((T, N))

    intercept_unscaled = betas[0, :, :] - adjustment  # shape = (T, 1)

    # 3) Keep categorical slopes unchanged
    cat_betas = betas[q+1 :, :, :]  # shape = ((K-1-q), T, 1)
    if cat_betas.ndim == 2:
        cat_betas = cat_betas.reshape(1, T, N)

    # 4) Re‐assemble
    intercept_unscaled = intercept_unscaled.reshape(1, T, N)
    betas_unscaled = np.concatenate(
        [intercept_unscaled, slopes_unscaled, cat_betas], axis=0
    )  # shape = (K, T, 1)

    return betas_unscaled


def ewm_standardize(df: pd.DataFrame, features: list, target: str,
                    halflife: float, burnin_steps: int):
    """
    EWMA scaling of numerical features, add intercepts, and reshape:
      - Mask out the first `burnin_steps` of the EWM so they don’t bias.
      - Backfill the masked rows from the first valid EWM.

    Args:
        df: pandas DataFrame (length T)
        features: list of feature column names (some numeric, some categorical)
        target: name of target column
        halflife: float, half‐life parameter for pandas’ ewm
        burnin_steps: int, number of initial rows to mask before backfill

    Returns:
        x: np.ndarray of shape (K, T, 1),
           row 0 = intercept,
           rows 1..q = (X_numeric - ewm_mean)/ewm_std (after burn‐in/backfill),
           rows q+1.. = categorical columns (unchanged).
        y: np.ndarray of shape (T, 1) (target, unchanged)
        ewm_means: np.ndarray of shape (q, T)
        ewm_stds:  np.ndarray of shape (q, T)
    """
    # 0) Check all features exist
    missing = [c for c in features if c not in df.columns]
    if missing:
        raise ValueError(f"Requested features not found in DataFrame: {missing}")

    T = df.shape[0]
    N = 1

    # 1) Partition features into numeric vs. categorical (preserve order)
    numeric_cols = [c for c in features if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols     = [c for c in features if not pd.api.types.is_numeric_dtype(df[c])]
    cols_order   = numeric_cols + cat_cols  # for final concatenation

    q = len(numeric_cols)

    # 2) Compute EWMA means/stds as DataFrames (shape: (T, q))
    if q > 0:
        ewm_means_df = df[numeric_cols].ewm(halflife=halflife, adjust=False).mean()
        ewm_stds_df  = df[numeric_cols].ewm(halflife=halflife, adjust=False).std() + 1e-8
    else:
        # No numeric features: create empty DataFrames with shape (T, 0)
        ewm_means_df = pd.DataFrame(index=df.index, columns=[])
        ewm_stds_df  = pd.DataFrame(index=df.index, columns=[])

    # 3) Mask the first `burnin_steps` rows for both means and stds
    if q > 0:
        ewm_means_df.iloc[:burnin_steps, :] = np.nan
        ewm_stds_df.iloc[:burnin_steps, :]  = np.nan

        # 4) Backfill masked rows
        ewm_means_df = ewm_means_df.bfill()
        ewm_stds_df  = ewm_stds_df.bfill()

    # 5) Convert back to NumPy and transpose to (q, T)
    if q > 0:
        means = ewm_means_df.to_numpy().T  # shape = (q, T)
        stds  = ewm_stds_df.to_numpy().T   # shape = (q, T)
    else:
        means = np.zeros((0, T))
        stds  = np.ones((0, T))

    # 6) Build scaled DataFrame for numeric columns (shape = (T, q))
    if q > 0:
        df_scaled_numeric = (df[numeric_cols] - ewm_means_df) / ewm_stds_df
    else:
        df_scaled_numeric = pd.DataFrame(index=df.index)

    # 7) Concatenate scaled numerics with categorical (unchanged)
    df_combined = pd.concat([df_scaled_numeric, df[cat_cols]], axis=1)
    df_new      = df_combined[cols_order]  # shape = (T, K-1)

    # 8) Build feature‐tensor x: shape = (K, T, 1)
    feat_array = df_new.to_numpy()               # shape = (T, K-1)
    feat_TN    = feat_array.T[:, :, np.newaxis]  # shape = (K-1, T, 1)
    inter      = np.ones((1, T, N))              # shape = (1, T, 1)
    x          = np.concatenate((inter, feat_TN), axis=0)  # shape = (K, T, 1)

    # 9) Build target vector (unchanged)
    y = df[target].to_numpy().reshape(T, N)  # shape = (T, 1)

    return x, y, means, stds


def ewm_unstandardize(betas: np.ndarray, ewm_means: np.ndarray, ewm_stds: np.ndarray):
    """
    Un‐scale time‐varying betas from EWM standardization.

    Args:
        betas:       np.ndarray of shape (K, T, 1),
                     [0] = scaled intercept,
                     [1:q+1] = scaled numeric slopes,
                     [q+1:]  = categorical (unchanged).
        ewm_means:   np.ndarray of shape (q, T)
        ewm_stds:    np.ndarray of shape (q, T)

    Returns:
        betas_unscaled: np.ndarray of shape (K, T, 1),
                        with numeric slopes and intercept un‐scaled to original units.
    """
    q, T = ewm_means.shape
    N = 1

    # 1) Un‐scale numeric slopes
    if q > 0:
        slopes_scaled    = betas[1 : q+1, :, :]                 # shape = (q, T, 1)
        slopes_unscaled  = slopes_scaled / ewm_stds.reshape(q, T, 1)  # shape = (q, T, 1)
    else:
        slopes_unscaled = np.zeros((0, T, N))

    # 2) Compute intercept adjustment
    if q > 0:
        temp = (ewm_means / ewm_stds).reshape(q, T, 1)  # shape = (q, T, 1)
        adjustment = np.sum(betas[1:q+1, :, :] * temp, axis=0)  # shape = (T, 1)
    else:
        adjustment = np.zeros((T, N))

    intercept_unscaled = betas[0, :, :] - adjustment  # shape = (T, 1)

    # 3) Keep categorical betas unchanged
    cat_betas = betas[q+1 :, :, :]  # shape = ((K-1-q), T, 1)
    if cat_betas.ndim == 2:
        cat_betas = cat_betas.reshape(1, T, N)

    # 4) Re‐assemble
    intercept_unscaled = intercept_unscaled.reshape(1, T, N)
    betas_unscaled = np.concatenate(
        [intercept_unscaled, slopes_unscaled, cat_betas],
        axis=0
    )  # shape = (K, T, 1)

    return betas_unscaled


# -------------------------------------------------------------------
# 2) Grid‐Search Over Tau & Decay Scale
# -------------------------------------------------------------------

def grid_search_param(x: np.ndarray, y: np.ndarray,
                      scaling: str, means: np.ndarray, stds: np.ndarray,
                      backshift: int):
    """
    Runs a quick grid search over tau (prior covariance scale) and decay_scale,
    using MSE on the original target to pick the best pair.  Always does predictions
    in the “scaled‐space” so that no unit mismatch happens.

    Args:
        x:        np.ndarray, shape = (K, T, 1), scaled input features
        y:        np.ndarray, shape = (T, 1), original target
        scaling:  "z", "ewm", or "identity"
        means:    np.ndarray (q,) if scaling=="z", or (q,T) if scaling=="ewm", else None
        stds:     np.ndarray (q,) or (q,T) or None
        backshift: int, number of time steps to lag coefficients before prediction

    Returns:
        best_tau:   float, best prior‐variance scale
        best_decay: float, best decay_scale
        mse_grid:   np.ndarray of shape (len(tau_vals), len(decay_vals)), MSE at each grid point
    """
    # Number of coefficient‐rows (intercept + all features)
    D = x.shape[0]

    # Define grid
    tau_vals   = np.logspace(-2, 2, num=5)   # e.g. [1e-2, 1e-1, 1, 10, 100]
    decay_vals = np.logspace(0, 4, num=5)    # e.g. [1, 10, 100, 1000, 10000]

    mse_grid = np.zeros((len(tau_vals), len(decay_vals)))
    best_mse  = np.inf
    best_tau  = None
    best_decay= None

    for i, tau in enumerate(tau_vals):
        for j, decay in enumerate(decay_vals):
            # 1) Fit Bayesian time‐series regression with decay
            prior_mean  = np.zeros(D)
            prior_covar = np.eye(D) * tau
            betas_temp = decay_regress(
                x=x, y=y,
                prior_mean=prior_mean,
                prior_covar=prior_covar,
                decay_scale=decay
            )  # shape = (D, T, 1)

            # 2) Back‐shift coefficients (still in scaled‐space)
            betas_lagged = np.roll(betas_temp, shift=backshift, axis=1)
            betas_lagged[:, :backshift, :] = np.nan

            # 3) Predict in scaled space:       (x is scaled, betas_lagged is scaled)
            yhat_temp = np.nansum(x * betas_lagged, axis=0)  # shape = (T, 1)

            # 4) Compute MSE vs original y
            resid = yhat_temp.flatten() - y.flatten()
            mse_temp = np.nanmean(resid**2)
            mse_grid[i, j] = mse_temp

            if mse_temp < best_mse:
                best_mse   = mse_temp
                best_tau   = tau
                best_decay = decay

    return best_tau, best_decay, mse_grid


# -------------------------------------------------------------------
# 3) Plotting / Diagnostics
# -------------------------------------------------------------------

def plot_model_diagnostics(y_flat: np.ndarray, yhat_flat: np.ndarray,
                           resid: np.ndarray, betas_unscaled: np.ndarray,
                           features: list, mse_grid: np.ndarray, grid_search: bool):
    """
    Generates diagnostic plots:
      1) Residuals over time
      2) Residual histogram
      3) Residuals vs. fitted
      4) Q-Q plot of residuals
      5) If grid_search=True, an MSE heatmap (tau × decay)
      6) Time‐series + histogram for each beta (unscaled)

    Args:
        y_flat:         np.ndarray of shape (T,), actual target
        yhat_flat:      np.ndarray of shape (T,), predicted target
        resid:          np.ndarray of shape (T,), residuals = yhat−y
        betas_unscaled: np.ndarray of shape (K, T, 1)
        features:       list of length (K−1) of feature names (no intercept)
        mse_grid:       np.ndarray of shape (len(tau_vals), len(decay_vals))
        grid_search:    bool, whether to plot the MSE heatmap
    """
    # 1) Residuals over time
    plt.figure(figsize=(6,4))
    plt.plot(resid, marker='o', linestyle='none')
    plt.xlabel("Time")
    plt.ylabel("Residual")
    plt.title("Residuals Over Time")
    plt.show()

    # 2) Residuals histogram
    plt.figure(figsize=(6,4))
    plt.hist(resid, bins=50, edgecolor='k')
    plt.xlabel("Residual")
    plt.ylabel("Frequency")
    plt.title("Residuals Histogram")
    plt.show()

    # 3) Residuals vs Fitted
    plt.figure(figsize=(6,4))
    plt.scatter(yhat_flat, resid, alpha=0.7)
    plt.axhline(0, color='gray', linewidth=1, linestyle='--')
    plt.xlabel("Fitted Values")
    plt.ylabel("Residuals")
    plt.title("Residuals vs Fitted")
    plt.show()

    # 4) Q‐Q plot of residuals
    fig = plt.figure(figsize=(6,4))
    ax4 = fig.add_subplot(111)
    sm.qqplot(resid, line='s', ax=ax4)
    ax4.set_title("Q-Q Plot of Residuals")
    plt.show()

    # 5) MSE Heatmap from grid‐search (if applicable)
    if grid_search and mse_grid is not None:
        # Assume tau_vals and decay_vals were defined inside grid_search_param;
        # we can recreate them exactly here for axis ticks:
        tau_vals   = np.logspace(-2, 2, num=mse_grid.shape[0])
        decay_vals = np.logspace(0, 4, num=mse_grid.shape[1])

        plt.figure(figsize=(8,6))
        plt.imshow(mse_grid.T, origin='lower', aspect='auto',
                   extent=[tau_vals[0], tau_vals[-1], decay_vals[0], decay_vals[-1]])
        plt.colorbar(label='MSE')
        plt.xlabel('Tau (prior‐variance)')
        plt.ylabel('Decay Scale')
        plt.title('MSE Heatmap')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()

    # 6) Plot each beta‐time series + histogram (unscaled)
    names = ["Intercept"] + features
    K = betas_unscaled.shape[0]

    for i in range(K):
        b_i = betas_unscaled[i].flatten()  # shape = (T,)
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
        ax1.plot(b_i, marker='o', linestyle='-')
        ax1.set_title(f"{names[i]} Over Time")
        ax1.set_xlabel("Time")
        ax1.set_ylabel("Beta Value")

        ax2.hist(b_i, bins=50, edgecolor='k')
        ax2.set_title(f"{names[i]} Histogram")
        ax2.set_xlabel("Beta Value")
        ax2.set_ylabel("Frequency")

        plt.tight_layout()
        plt.show()


# -------------------------------------------------------------------
# 4) Top‐Level Pipeline Function
# -------------------------------------------------------------------

def decay_pipeline(df: pd.DataFrame, features: list, target: str,
                   scaling: str = "z", scale_kwargs: dict = None,
                   backshift: int = 48, tau: float = 10, decay_scale: float = 2880,
                   grid_search: bool = True, plot: bool = False):
    """
    Wrapper that:
      0) Validates inputs,
      1) Scales & shapes data (z, ewm, or identity),
      2) Optionally grid‐searches over (tau, decay_scale),
      3) Fits final decay_regress model,
      4) Un‐scales betas for interpretability (if requested),
      5) Produces predictions & metrics (MSE, R2),
      6) (Optional) Plots diagnostics.

    Args:
        df:          pandas DataFrame (length T)
        features:    list of feature column names
        target:      string name of target column
        scaling:     "z", "ewm", or "identity"
        scale_kwargs: dict with keys "halflife" and "burnin_steps" if scaling=="ewm"
        backshift:   int, time‐lag for coefficients before making predictions
        tau:         float, default prior‐variance (if no grid_search)
        decay_scale: float, default decay_scale (if no grid_search)
        grid_search: bool, whether to run grid_search over (tau, decay_scale)
        plot:        bool, whether to call plot_model_diagnostics at the end

    Returns:
        dict with keys:
          "prior_mean":   np.ndarray of shape (K,)
          "prior_covar":  np.ndarray of shape (K,K)  (covariance matrix)
          "decay_scale":  float, the selected decay scale
          "betas":        np.ndarray of shape (K, T, 1)  (lagged, unscaled if appropriate)
          "yhat":         np.ndarray of shape (T, 1)       (predictions in original space)
          "resid":        np.ndarray of shape (T,)         (yhat_flat - y_flat)
          "mse":          float
          "r2":           float
    """
    # 0) Ensure features exist
    missing = [c for c in features if c not in df.columns]
    if missing:
        raise ValueError(f"Requested features not found in DataFrame: {missing}")

    # 1) Scale & Shape
    if scaling == "z":
        x, y, means, stds = z_standardize(df, features, target)
    elif scaling == "ewm":
        if not isinstance(scale_kwargs, dict):
            raise ValueError("When scaling='ewm', scale_kwargs must be a dict with keys 'halflife' and 'burnin_steps'.")
        hl = scale_kwargs.get("halflife")
        bs = scale_kwargs.get("burnin_steps")
        if hl is None or bs is None:
            raise ValueError("scale_kwargs must contain 'halflife' and 'burnin_steps'.")
        x, y, means, stds = ewm_standardize(df, features, target, hl, bs)
    else:
        x, y, means, stds = identity_standardize(df, features, target)

    D = x.shape[0]  # number of coefficient‐rows (intercept + features)

    # 2) Set Priors and optionally run grid search
    if grid_search:
        best_tau, best_decay, mse_grid = grid_search_param(x, y, scaling, means, stds, backshift)
        prior_mean  = np.zeros(D)
        prior_covar = np.eye(D) * best_tau
        decay_scale = best_decay
    else:
        best_tau    = tau
        best_decay  = decay_scale
        mse_grid    = None
        prior_mean  = np.zeros(D)
        prior_covar = np.eye(D) * tau

    # 3) Fit final regression (in scaled‐space)
    betas = decay_regress(
        x=x, y=y,
        prior_mean=prior_mean,
        prior_covar=prior_covar,
        decay_scale=decay_scale
    )  # shape = (D, T, 1)

    # 4) Un‐scale betas for interpretability (optional)
    if scaling == "z":
        betas_unscaled = z_unstandardize(betas, means, stds)
    elif scaling == "ewm":
        betas_unscaled = ewm_unstandardize(betas, means, stds)
    else:
        betas_unscaled = betas  # identity: no scaling applied

    # 5) Backshift & Predict (still in scaled‐space, because y was never scaled)
    betas_lagged = np.roll(betas, shift=backshift, axis=1)
    betas_lagged[:, :backshift, :] = np.nan
    yhat = np.nansum(x * betas_lagged, axis=0)  # shape = (T, 1)

    # 6) Compute metrics (all in original y‐units, since y was never scaled):
    y_flat    = y.flatten()
    yhat_flat = yhat.flatten()
    resid     = yhat_flat - y_flat
    mse       = np.nanmean(resid**2)
    ss_res    = np.nansum((y_flat - yhat_flat)**2)
    ss_tot    = np.nansum((y_flat - np.nanmean(y_flat))**2)
    r2        = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan

    # 7) Plot diagnostics if requested
    if plot:
        plot_model_diagnostics(y_flat, yhat_flat, resid, betas_unscaled, features, mse_grid, grid_search)

    return {
        "prior_mean":   prior_mean,
        "prior_covar":  prior_covar,
        "decay_scale":  decay_scale,
        "betas":        betas_lagged,
        "yhat":         yhat,
        "resid":        resid,
        "mse":          mse,
        "r2":           r2
    }


### Data Pre Processing Steps 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector


# 1) ---------------- LagFeaturesTransformer ----------------
class LagFeaturesTransformer(BaseEstimator, TransformerMixin):
    """
    - For every column in X:
      • If column name contains "_f" → keep it untouched.
      • If column is dtype 'category'     → keep it untouched.
      • Otherwise, if column is numeric (including "imbalance"):
          • create a lagged version "<col>_lag1" = df[col].shift(1)
          • drop the original "<col>"
      • Any non-numeric, non-forecast, non-categorical column is dropped.
    """
    def __init__(self, lag: int = 1):
        self.lag = lag

    def fit(self, X: pd.DataFrame, y=None):
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        to_drop = []
        lagged_data = {}

        for col in df.columns:
            if "_f" in col:
                # forecast columns: keep as-is
                continue

            if pd.api.types.is_categorical_dtype(df[col]):
                # already categorical: keep as-is
                continue

            if pd.api.types.is_numeric_dtype(df[col]):
                # numeric & not a forecast: create lag and drop original
                lagged_name = f"{col}_lag{self.lag}"
                lagged_data[lagged_name] = df[col].shift(self.lag)
                to_drop.append(col)
            else:
                # non-numeric, non-forecast, non-categorical: drop
                to_drop.append(col)

        # Insert all lagged columns
        for lagged_name, series in lagged_data.items():
            df[lagged_name] = series

        # Drop the originals
        df = df.drop(columns=to_drop)
        return df


# 2) ---------------- CategoricalCaster ----------------
class CategoricalCaster(BaseEstimator, TransformerMixin):
    """
    - Given a list of column names, casts df[col] → pd.Categorical.
      Leaves everything else untouched.
    """
    def __init__(self, categorical_columns: list[str]):
        self.categorical_columns = categorical_columns

    def fit(self, X: pd.DataFrame, y=None):
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        for col in self.categorical_columns:
            if col in df.columns:
                df[col] = df[col].astype("category")
        return df


# 3) ------------- PercentileCategorizer ----------------
class PercentileCategorizer(BaseEstimator, TransformerMixin):
    """
    - cols_to_cat: list of numeric column names to bin by percentile.
    - percentile: float in (0,1), e.g. 0.90 for 90th percentile.
    - new_suffix: suffix for each new category column (e.g. "_pctcat").
    
    In fit(X):
      • For each col in cols_to_cat, compute threshold = X[col].quantile(percentile).
    In transform(X):
      • For each col, create "<col>_pctcat" = "high" if X[col] >= threshold, else "normal".
      • Cast each new "<col>_pctcat" to dtype 'category'.
      • Leave original numeric columns intact.
    """
    def __init__(
        self,
        cols_to_cat: list[str],
        percentile: float = 0.9,
        new_suffix: str = "_pctcat"
    ):
        self.cols_to_cat = cols_to_cat
        self.percentile = percentile
        self.new_suffix = new_suffix
        self.thresholds_: dict[str, float] = {}

    def fit(self, X: pd.DataFrame, y=None):
        for col in self.cols_to_cat:
            if col not in X.columns:
                raise ValueError(f"Column '{col}' not found in X during PercentileCategorizer.fit().")
            if not pd.api.types.is_numeric_dtype(X[col]):
                raise ValueError(f"Column '{col}' must be numeric to create percentile categories.")
            self.thresholds_[col] = float(X[col].quantile(self.percentile))
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        for col, thresh in self.thresholds_.items():
            new_col = f"{col}{self.new_suffix}"
            df[new_col] = np.where(df[col] >= thresh, "high", "normal")
            df[new_col] = pd.Categorical(df[new_col], categories=["normal", "high"])
        return df


# 4) ------------- The Full Feature-Engineering Pipeline -------------
def build_feature_pipeline(
    categorical_columns: list[str],
    percentile_columns: list[str],
    percentile: float = 0.9,
    lag: int = 1
) -> Pipeline:
    """
    Returns an sklearn Pipeline that:
      1) Lag all numeric columns not containing "_f" (and drop the originals).
      2) Cast any pre-existing columns in `categorical_columns` to dtype 'category'.
      3) For each column in `percentile_columns`, create a new "<col>_pctcat"
         categorical column based on the given percentile.
      4) Run a ColumnTransformer that:
         - Imputes (median) and scales (StandardScaler) all remaining numeric columns.
         - One-hot-encodes all categorical columns (including newly created "<col>_pctcat").
    """
    # Step 1: lag & drop
    lag_step = ("lag_features", LagFeaturesTransformer(lag=lag))

    # Step 2: cast existing categories
    cast_cat_step = ("cast_categoricals", CategoricalCaster(categorical_columns))

    # Step 3: create percentile-based categories
    pct_step = ("percentile_categories", PercentileCategorizer(
        cols_to_cat=percentile_columns,
        percentile=percentile,
        new_suffix="_pctcat"
    ))

    # Step 4: final ColumnTransformer
    numeric_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_pipeline = Pipeline([
        ("onehot", OneHotEncoder(sparse=False, drop="if_binary"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            # All remaining numeric columns
            ("num", numeric_pipeline, make_column_selector(dtype_include="number")),
            # All remaining categorical columns
            ("cat", categorical_pipeline, make_column_selector(dtype_include="category")),
        ],
        remainder="drop"
    )

    pipeline = Pipeline([
        lag_step,
        cast_cat_step,
        pct_step,
        ("preprocessor", preprocessor),
    ])

    return pipeline


### Rolling regression 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def compute_cumulative_r2(y_actual: pd.Series, y_pred: pd.Series, start_idx: int = 1488) -> pd.Series:
    """
    Compute cumulative R² = 1 – SS_res/SS_tot for each t ≥ start_idx.
    Returns a pandas Series aligned with the original index of y_actual.
    """
    # Align indexes
    y_actual_aligned = y_actual.reset_index(drop=True)
    y_pred_aligned   = y_pred.reset_index(drop=True)
    n = len(y_actual_aligned)

    cum_r2 = pd.Series(index=y_actual_aligned.index, dtype=float)
    running_ss_res = 0.0

    for t in range(start_idx, n):
        # Update SS_res with the new residual at time t
        residual_t = y_actual_aligned.iloc[t] - y_pred_aligned.iloc[t]
        running_ss_res += residual_t ** 2

        # Compute SS_tot over [start_idx … t]
        window_vals = y_actual_aligned.iloc[start_idx:t+1]
        mean_window = window_vals.mean()
        ss_tot = ((window_vals - mean_window) ** 2).sum()

        # Cumulative R² at time t
        if ss_tot != 0:
            cum_r2.iloc[t] = 1 - (running_ss_res / ss_tot)
        else:
            cum_r2.iloc[t] = np.nan

    cum_r2.index = y_actual.index
    return cum_r2


def compute_rolling_r2(
    y_actual: pd.Series,
    y_pred:   pd.Series,
    start_idx: int = 1488,
    window:    int = 336,
    method:   str = 'standard'
) -> pd.Series:
    """
    Compute rolling‐window R² for t ≥ start_idx + window – 1.
    Options:
      - method='standard':     R² = 1 – SS_res/SS_tot in each window
      - method='regression':   R² = [corr(y, ŷ)]² in each window
    Returns a pandas Series aligned with the original index of y_actual.
    """
    y_actual_aligned = y_actual.reset_index(drop=True)
    y_pred_aligned   = y_pred.reset_index(drop=True)
    n = len(y_actual_aligned)

    roll_r2 = pd.Series(index=y_actual_aligned.index, dtype=float)

    for t in range(start_idx + window - 1, n):
        y_win    = y_actual_aligned.iloc[t - window + 1 : t + 1]
        yhat_win = y_pred_aligned.iloc[t - window + 1 : t + 1]

        if method == 'standard':
            ss_res = ((y_win - yhat_win) ** 2).sum()
            mean_win = y_win.mean()
            ss_tot = ((y_win - mean_win) ** 2).sum()
            if ss_tot != 0:
                roll_r2.iloc[t] = 1 - (ss_res / ss_tot)
            else:
                roll_r2.iloc[t] = np.nan

        elif method == 'regression':
            corr_val = y_win.corr(yhat_win)
            roll_r2.iloc[t] = corr_val ** 2 if not np.isnan(corr_val) else np.nan

    roll_r2.index = y_actual.index
    return roll_r2


# ===== Example Usage =====
# (Uncomment & replace with your actual data)
#
# y_actual_series = tokyo_variables["Imbalance_Minus_Spot"]
# y_pred_series   = yhat_output1_pd
#
# # 1) Cumulative R² starting at index 1488:
# cum_r2_series = compute_cumulative_r2(
#     y_actual_series,
#     y_pred_series,
#     start_idx=1488
# )
#
# # 2) Rolling R² with 336‐period windows, using regression‐based R²:
# roll_r2_series = compute_rolling_r2(
#     y_actual_series,
#     y_pred_series,
#     start_idx=1488,
#     window=336,
#     method='regression'
# )
#
# # 3) Plot both metrics:
# plt.figure(figsize=(12,4))
# plt.plot(cum_r2_series.index, cum_r2_series.values, label='Cumulative R²')
# plt.axvline(x=cum_r2_series.index[1488], color='k', linestyle='--', label='Start (1488)')
# plt.ylim(-1, 1)
# plt.title('Cumulative R² (from index 1488 onward)')
# plt.legend()
# plt.grid(True)
# plt.show()
#
# plt.figure(figsize=(12,4))
# plt.plot(roll_r2_series.index, roll_r2_series.values, label='Rolling R² (window=336)')
# plt.axvline(x=roll_r2_series.index[1488 + 336 - 1],
#             color='k', linestyle='--',
#             label='First full window end')
# plt.ylim(-1, 1)
# plt.title('Rolling R² (336‐period windows, from index 1488 onward)')
# plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_two_r2(
    y_actual: pd.Series,
    y_pred:   pd.Series,
    start_idx: int = 1488,
    window:    int = 336,
    method:   str = 'standard'
) -> None:
    """
    Plots two R²‐series for a decay regression model:
      1) Cumulative R² from index = start_idx onward
      2) Rolling‐window R² (window length = window) for indices ≥ (start_idx + window – 1)

    Parameters
    ----------
    y_actual : pd.Series
        The observed target time series. Its index is used for plotting.
    y_pred : pd.Series
        The predicted values from your decay regression model.
        Must have the same length (and corresponding time alignment) as y_actual.
    start_idx : int, default 1488
        The “burn‐in” index: the first point at which we begin computing R².
    window : int, default 336
        The length of the sliding window used for the rolling‐R² calculation.
    method : {'standard', 'regression'}, default 'standard'
        - 'standard':       R² = 1 – (SS_res / SS_tot) in each window (and cumulatively)
        - 'regression':     R² = [corr(y_window, ŷ_window)]²  
           i.e. regressing predicted on actual in each window or cumulatively.

    This function does not return anything; it produces two matplotlib plots.
    """

    # 1) Make sure y_actual and y_pred have the same length & index
    if len(y_actual) != len(y_pred):
        raise ValueError("y_actual and y_pred must have the same length.")

    orig_index = y_actual.index

    # Convert to numpy arrays for fast slicing; we’ll re‐apply orig_index later:
    y_true_vals = y_actual.reset_index(drop=True).astype(float).values
    y_pred_vals = y_pred.reset_index(drop=True).astype(float).values
    n = len(y_true_vals)

    # -----------------------------
    # 2) Compute cumulative R²
    # -----------------------------
    cum_r2 = np.full(n, np.nan, dtype=float)

    if method == 'standard':
        # Keep a running sum of squared residuals from start_idx onward
        running_ss_res = 0.0
        for t in range(start_idx, n):
            resid_t = (y_true_vals[t] - y_pred_vals[t])
            running_ss_res += resid_t**2

            # Compute SS_tot over [start_idx .. t]
            y_window = y_true_vals[start_idx : t+1]
            mean_window = y_window.mean()
            ss_tot = ((y_window - mean_window)**2).sum()

            if ss_tot != 0:
                cum_r2[t] = 1.0 - (running_ss_res / ss_tot)
            else:
                cum_r2[t] = np.nan

    elif method == 'regression':
        # At each t ≥ start_idx, do corr(true[start_idx:t], pred[start_idx:t])²
        for t in range(start_idx, n):
            y_win    = y_true_vals[start_idx : t+1]
            yhat_win = y_pred_vals[start_idx : t+1]
            if len(y_win) < 2:
                cum_r2[t] = np.nan
            else:
                s_true = pd.Series(y_win)
                s_pred = pd.Series(yhat_win)
                corr_val = s_true.corr(s_pred)
                cum_r2[t] = corr_val**2 if not np.isnan(corr_val) else np.nan
    else:
        raise ValueError("method must be 'standard' or 'regression'.")

    cum_r2_series = pd.Series(cum_r2, index=orig_index)

    # -----------------------------
    # 3) Compute rolling‐window R² (window length = window)
    # -----------------------------
    roll_r2 = np.full(n, np.nan, dtype=float)

    if method == 'standard':
        for t in range(start_idx + window - 1, n):
            y_win    = y_true_vals[t - window + 1 : t+1]
            yhat_win = y_pred_vals[t - window + 1 : t+1]

            ss_res = ((y_win - yhat_win)**2).sum()
            mean_win = y_win.mean()
            ss_tot = ((y_win - mean_win)**2).sum()
            if ss_tot != 0:
                roll_r2[t] = 1.0 - (ss_res / ss_tot)
            else:
                roll_r2[t] = np.nan

    else:  # method == 'regression'
        for t in range(start_idx + window - 1, n):
            y_win    = y_true_vals[t - window + 1 : t+1]
            yhat_win = y_pred_vals[t - window + 1 : t+1]
            s_true = pd.Series(y_win)
            s_pred = pd.Series(yhat_win)
            corr_val = s_true.corr(s_pred)
            roll_r2[t] = corr_val**2 if not np.isnan(corr_val) else np.nan

    roll_r2_series = pd.Series(roll_r2, index=orig_index)

    # -----------------------------
    # 4) Produce the two plots
    # -----------------------------
    # (a) Cumulative R²
    plt.figure(figsize=(12, 4))
    plt.plot(
        cum_r2_series.index,
        cum_r2_series.values,
        label=f"Cumulative R² ({method})",
        linewidth=1.5
    )
    plt.axvline(
        x=cum_r2_series.index[start_idx],
        color='k',
        linestyle='--',
        label=f"Start index = {start_idx}"
    )
    plt.ylim(-1, 1)
    plt.title(f"Cumulative R² ({method}, from index {start_idx})")
    plt.xlabel("Time Index")
    plt.ylabel("R²")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # (b) Rolling‐window R²
    first_full_window_idx = start_idx + window - 1
    plt.figure(figsize=(12, 4))
    plt.plot(
        roll_r2_series.index,
        roll_r2_series.values,
        label=f"Rolling R² ({method}, window={window})",
        linewidth=1.5
    )
    plt.axvline(
        x=roll_r2_series.index[first_full_window_idx],
        color='k',
        linestyle='--',
        label=f"First full window ends at index = {first_full_window_idx}"
    )
    plt.ylim(-1, 1)
    plt.title(
        f"Rolling R² ({method}, {window}-period windows, from index {start_idx})"
    )
    plt.xlabel("Time Index")
    plt.ylabel("R²")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
